In [52]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data

import pickle
import numpy as np
import pandas as pd

np.set_printoptions(threshold=np.inf)
pd.set_option('display.max_colwidth', None)

In [53]:
# HYPERPARAMETERS ---

epochs = 10
batch_size = 100
learning_rate = 0.001

In [54]:
with open("./datasets/mnist/train-images-flatten.pkl", "rb") as f:
  train_x = pickle.load(f)

with open("./datasets/mnist/train-labels.pkl", "rb") as f:
  train_t = pickle.load(f)

with open("./datasets/mnist/test-images-flatten.pkl", "rb") as f:
  test_x = pickle.load(f)

with open("./datasets/mnist/test-labels.pkl", "rb") as f:
  test_t = pickle.load(f)
  
print(train_x.shape)

(60000, 784)


In [55]:
class Dataset(data.Dataset):
  def __init__(self, data, label) :
    self.data = data
    self.label = label

  def __len__(self) :
    return len(self.data)

  def __getitem__(self, idx) :
    data = torch.FloatTensor(self.data[idx])
    label = torch.FloatTensor(self.label[idx])

    return data, label

In [56]:
train_dataset = Dataset(train_x, train_t)
test_dataset = Dataset(test_x, test_t)

train, validation = torch.utils.data.random_split(train_dataset, [0.9, 0.1])

train_loader = data.DataLoader(train, batch_size, shuffle=True)
validation_loader = data.DataLoader(validation, batch_size, shuffle=True)
test_loader = data.DataLoader(test_dataset, batch_size)

In [ ]:
class Model(nn.Module):
  def __init__(self):
    super().__init__()
    self.rnn = nn.RNN(28*28, 100, 1, nonlinearity='relu', batch_first=True)
    self.layers = nn.Sequential(
      nn.Linear(100, 10)
    )

  def forward(self, x):
    y, hn = self.rnn(x)
    y = self.layers(y)
    return y
  
model = Model()
model

Model(
  (rnn): RNN(784, 100, num_layers=5, batch_first=True)
  (layers): Sequential(
    (0): Linear(in_features=100, out_features=10, bias=True)
  )
)

In [58]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), learning_rate)

for epoch in range(epochs):
  for i, (x, t) in enumerate(train_loader):
    optimizer.zero_grad()
    y = model(x)
    loss_train = criterion(y, t)
    loss_train.backward()
    optimizer.step()

    if i % 10 == 0:
      for x, t in validation_loader:
        y = model(x)
        loss_validation = criterion(y, t)

      print(f"Epoch #{epoch}, Batch #{i}, Train Loss: {loss_train.item():.4f}, Validation Loss: {loss_validation.item():.4f}")

Epoch #0, Batch #0, Train Loss: 2.3007, Validation Loss: 2.3077
Epoch #0, Batch #10, Train Loss: 2.2867, Validation Loss: 2.2700
Epoch #0, Batch #20, Train Loss: 2.1413, Validation Loss: 2.0721
Epoch #0, Batch #30, Train Loss: 1.6439, Validation Loss: 1.5722
Epoch #0, Batch #40, Train Loss: 1.3682, Validation Loss: 1.4241
Epoch #0, Batch #50, Train Loss: 1.1721, Validation Loss: 1.3392
Epoch #0, Batch #60, Train Loss: 1.3817, Validation Loss: 1.1258
Epoch #0, Batch #70, Train Loss: 1.0462, Validation Loss: 1.0477
Epoch #0, Batch #80, Train Loss: 1.2601, Validation Loss: 0.9941
Epoch #0, Batch #90, Train Loss: 0.9834, Validation Loss: 0.8770
Epoch #0, Batch #100, Train Loss: 0.8066, Validation Loss: 0.6819
Epoch #0, Batch #110, Train Loss: 0.6940, Validation Loss: 0.7530
Epoch #0, Batch #120, Train Loss: 0.7374, Validation Loss: 0.7338
Epoch #0, Batch #130, Train Loss: 0.5382, Validation Loss: 0.7074
Epoch #0, Batch #140, Train Loss: 0.7957, Validation Loss: 0.5829
Epoch #0, Batch #150,

In [60]:
model.eval()
total = 0
correct = 0

for x, t in test_loader:
  y = model(x)
  y = torch.argmax(y, dim=1)
  t = torch.argmax(t, dim=1)

  total += t.size(0)
  correct += (y == t).sum().item()

test_accuracy = 100 * correct / total
print(f"Test Accuracy: {test_accuracy:.2f}%")

Test Accuracy: 97.28%
